In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# setup CUDA_VISIBLE DEVICES for titan.sci.utah.edu
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [ ]:
#Import libraries
import torch
import numpy as np
import matplotlib as plt

from PIL import Image

from diffusers import StableDiffusionPipeline                                                                                                                                                                                                 
from diffusers import DPMSolverMultistepScheduler
from diffusers import AutoencoderKL

In [ ]:
#model_id = "runwayml/stable-diffusion-v1-5"  
model_id = "prompthero/openjourney"
#model_id = "stabilityai/stable-diffusion-2-1"
#vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")

In [ ]:
#pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
#prompt = "black and white portrait photo of an old warrior chief"
#prompt += " 50mm portrait photography, hard rim lighting photography--beta --ar 2:3  --beta --upbeta"

# prompt = "futuristic hospital room mixing  AI and medicine, showing humanoid robot, MRI scan, glass display, computers and high-tech equipment, deep blue on dark background, \
# ultra realistic, concept art, intricate details, highly detailed, octane render, \
# 8k, unreal engine, sharp focus, volumetric lighting unreal engine, \
# trending on art station"

prompt = "3d rendering showing a distant human brain, with glowing neuron connections \
forming a complex network of thoughts and memories. \
deep blue --ar 9:16 \
ultra realistic, concept art, intricate details, highly detailed, octane render, \
8k, unreal engine, sharp focus, volumetric lighting unreal engine, \
mdjrny-v4 style"


In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
# Change autoencoder
#pipe.vae = vae
# Run cross-attention sequentially - reducing memory requirements
pipe.enable_attention_slicing()

# Updating scheduler 
#pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
def get_inputs(batch_size=1):                                                                                                                                                                                                                 
  generator = [torch.Generator("cuda").manual_seed(i) for i in range(batch_size)]                                                                                                                                                             
  prompts = batch_size * [prompt]                                                                                                                                                                                                             
  num_inference_steps = 50                                                                                                                                                                                                                    

  return {"prompt": prompts, "generator": generator, "num_inference_steps": num_inference_steps}                                                                                                                                              

In [ ]:
def image_grid(imgs, rows=2, cols=4):                                                                                                                                                                                                         
    w, h = imgs[0].size
    print('w-h',[w,h])
    grid = Image.new('RGB', size=(cols*w, rows*h))                                                                                                                                                                                            
                                                                                                                                                                                                                                              
    for i, img in enumerate(imgs):                                                                                                                                                                                                            
        grid.paste(img, box=(i%cols*w, i//cols*h))                                                                                                                                                                                            
    return grid

In [ ]:
images = pipe(**get_inputs(batch_size=8)).images                                                                                                                                                                                              
image_grid(images)

In [ ]:
images[0].size

In [ ]:
display(images[4])

In [ ]:
images[4].save('3DBrain_Visualization4.png')